# Manipulación y Transformación de Datos con Pandas
**Curso:** Fundamentos de Programación y Analítica de Datos con Python  

**Duración estimada del bloque:** 2 horas (Sección aplicada dentro de la Sesión 5)  

**Objetivos específicos**
- Importar y exportar datasets en formatos comunes (CSV y Excel) utilizando Pandas.
- Seleccionar y filtrar subconjuntos de datos con `loc`, `iloc` y condiciones booleanas compuestas.
- Aplicar operaciones de agregación y transformación con `groupby`, `agg` y `apply` para obtener métricas y nuevas variables.
- Adoptar buenas prácticas de limpieza, tipificación y documentación del proceso de manipulación de datos.

**Prerrequisitos**
- Conocimientos básicos de Python (tipos, listas, diccionarios, funciones).
- Nociones elementales de NumPy (arrays y operaciones vectorizadas).
- Conocimientos mínimos de estructura tabular (filas/columnas) y estadística descriptiva básica.

## Tema 1. Carga y Exportación de Datos (CSV / Excel)

### Definición
Operaciones de **entrada/salida (I/O)** para leer datos desde archivos externos (CSV, Excel) hacia un `DataFrame` de Pandas y para **persistir** resultados de procesamiento con `to_csv` / `to_excel`.

### Importancia en programación y analítica de datos
- Los datos en entornos reales provienen de fuentes heterogéneas; la **ingesta fiable** es el primer paso del pipeline.
- La capacidad de **persistir** resultados garantiza reproducibilidad, intercambio e integración con otras herramientas (BI, hojas de cálculo, SQL).
- Parámetros de lectura correctos (separador, codificación, tipos) evitan errores de calidad de datos desde el inicio.

### Buenas prácticas profesionales y errores comunes
- **Buenas prácticas:** especificar parámetros relevantes (`sep`, `encoding`, `dtype`, `usecols`, `parse_dates`), validar con `.head()`, `.info()`, y documentar supuestos.
- **Errores comunes:** depender del autoinferido de tipos sin validación; no controlar separadores o codificaciones; exportar con índices no deseados.

In [6]:

# TODO: Ejemplo en Python: lectura y escritura segura con Pandas
import pandas as pd
from io import StringIO
from pathlib import Path

#* Simulación de un CSV contenido en memoria
csv_text = """id,producto,categoria,precio,fecha,unidades
1,Auriculares,Audio,35.5,2024-01-02,3
2,Teclado,Periféricos,29.9,2024-01-03,2
3,Monitor,Displays,199.0,2024-01-03,1
4,Mouse,Periféricos,15.0,2024-01-04,5
5,Barra de sonido,Audio,120.0,2024-01-05,1
"""

# Leer CSV desde memoria ( equivalente a leer desde un archivo: pd.read_csv('archivo.csv') )
df = pd.read_csv(StringIO(csv_text), sep=',', encoding='utf-8', parse_dates=['fecha'])

# Inspección Rápida
print("Dimensiones del Dataframe: ", df.shape)
print(df.head())
print("Tipo de datos inferidos:\n", df.dtypes)

out_dir = Path('./data')
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = out_dir / 'productos.csv'
xlsx_path = out_dir / 'productos.xlsx'

# Exportar en archivos limpios
df.to_csv(csv_path, index=False, encoding='utf-8')
df.to_excel(xlsx_path, index=False, engine='openpyxl')



Dimensiones del Dataframe:  (5, 6)
   id         producto    categoria  precio      fecha  unidades
0   1      Auriculares        Audio    35.5 2024-01-02         3
1   2          Teclado  Periféricos    29.9 2024-01-03         2
2   3          Monitor     Displays   199.0 2024-01-03         1
3   4            Mouse  Periféricos    15.0 2024-01-04         5
4   5  Barra de sonido        Audio   120.0 2024-01-05         1
Tipo de datos inferidos:
 id                    int64
producto             object
categoria            object
precio              float64
fecha        datetime64[ns]
unidades              int64
dtype: object


## Tema 2. Selección y Filtrado: `loc`, `iloc` y Condiciones

### Definición
**Selección** de filas/columnas por **etiqueta** (`loc`) o por **posición** (`iloc`) y **filtrado** mediante condiciones booleanas simples o compuestas.

### Importancia en programación y analítica de datos
- Permite construir subconjuntos para **análisis focalizado**, limpieza dirigida y verificación de hipótesis.
- Es la base para preparar datasets de modelado y para aplicar transformaciones específicas por segmentos.

### Buenas prácticas profesionales y errores comunes
- **Buenas prácticas:** preferir `loc` cuando se conocen etiquetas; usar máscaras booleans legibles; encadenar condiciones con `&`, `|` y paréntesis.
- **Errores comunes:** olvidar paréntesis al combinar condiciones; confundir `loc` (incluye extremo superior en slices) con `iloc` (excluye extremo superior).

In [12]:

# TODO: Ejemplo en Python: loc, iloc y filtros compuestos

df = pd.read_csv("./data/productos.csv", parse_dates=['fecha'])

# Selección por etiquetas de columnas con loc
subset_loc = df.loc[:, ['producto', 'categoria', 'precio']]
print("Subset usando loc:\n", subset_loc.head())

# Filtrado adicional
mask = (df["categoria"] == "Periféricos") & (df["precio"] < 20)
filtrados = df[mask]
print("Productos filtrados:\n", filtrados)

filtrado_fecha = df[df["fecha"] >= "2024-01-04"]
print("Productos filtrados por fecha:\n", filtrado_fecha)


Subset usando loc:
           producto    categoria  precio
0      Auriculares        Audio    35.5
1          Teclado  Periféricos    29.9
2          Monitor     Displays   199.0
3            Mouse  Periféricos    15.0
4  Barra de sonido        Audio   120.0
Productos filtrados:
    id producto    categoria  precio      fecha  unidades
3   4    Mouse  Periféricos    15.0 2024-01-04         5
Productos filtrados por fecha:
    id         producto    categoria  precio      fecha  unidades
3   4            Mouse  Periféricos    15.0 2024-01-04         5
4   5  Barra de sonido        Audio   120.0 2024-01-05         1


## Tema 3. Operaciones de Agregación y Transformación: `groupby`, `agg`, `apply`

### Definición
- `groupby`: particiona el `DataFrame` en grupos según una o más columnas.
- `agg`: aplica funciones de **agregación** (p. ej., `sum`, `mean`, `count`) por grupo.
- `apply`: aplica una función arbitraria a cada grupo/serie para **transformaciones personalizadas**.

### Importancia en programación y analítica de datos
- Facilita el cálculo de **métricas resumidas** por categoría, periodo o segmento.
- Permite construir **features** y reglas de negocio (KPIs) a partir de datos brutos.

### Buenas prácticas profesionales y errores comunes
- **Buenas prácticas:** nombrar columnas agregadas con alias claros; validar cardinalidades; evitar `apply` cuando una operación vectorizada es suficiente.
- **Errores comunes:** no resetear el índice tras agregaciones; usar `apply` de forma innecesaria (penaliza rendimiento).

In [17]:

# TODO: Ejemplo en Python: groupby, agg y apply

df = pd.read_csv("./data/productos.csv", parse_dates=['fecha'])

# Métricas por categoría
agg_por_categoria = (
  df.groupby('categoria')
    .agg(
      total_unidades=('unidades', 'sum'),
      precio_promedio=('precio', 'mean'),
      productos_distintos=('producto', 'nunique')
    )
    .reset_index()
)
print("Métricas por categoría:\n", agg_por_categoria )
print("-" * 40, "\n")

# Función Personalizadas: rango( max - min ) por categoría
def rango(serie):
  return serie.max() - serie.min()

rango_precios = df.groupby('categoria')['precio'].apply(rango).reset_index(name='rango_precio')
print("Rango de precios por categoría:\n", rango_precios)

# Transformaciones por grupo: Normalizar unidades
def zscore(serie):
  return (serie - serie.mean()) / serie.std(ddof=0)

df["z_unidades_por_categoria"] = df.groupby('categoria')['unidades'].transform(zscore)
print(df[["categoria", "unidades", "z_unidades_por_categoria"]].head())

Métricas por categoría:
      categoria  total_unidades  precio_promedio  productos_distintos
0        Audio               4            77.75                    2
1     Displays               3           204.50                    1
2  Periféricos              11            23.30                    2
---------------------------------------- 

Rango de precios por categoría:
      categoria  rango_precio
0        Audio          84.5
1     Displays          11.0
2  Periféricos          14.9
     categoria  unidades  z_unidades_por_categoria
0        Audio         3                  1.000000
1  Periféricos         2                 -1.336306
2     Displays         1                 -1.000000
3  Periféricos         5                  1.069045
4        Audio         1                 -1.000000


# Ejercicios Integradores

A continuación se proponen ejercicios que integran carga, selección/filtrado y agregación/transformación. Se presentan **pistas** y, posteriormente, una **solución propuesta**.

## Ejercicio 1. Reporte de ventas por categoría y ticket promedio
**Contexto técnico:** Eres analista de Business Intelligence en una tienda en línea. Debes generar un **reporte semanal** para dirección con métricas por categoría que orienten decisiones de pricing y stock.

**Datos / entradas:** Utiliza un `DataFrame` con columnas `categoria`, `precio`, `unidades` y `fecha`. Puedes partir de un CSV simulado en memoria (similar a los ejemplos).

**Requerimientos:**
1. Calcular **ventas totales** por categoría (`precio * unidades`).
2. Calcular el **ticket promedio** por categoría (promedio de `precio` ponderado por `unidades` o simple, justifica tu elección).
3. Ordenar el resultado por ventas totales descendente.

**Criterios de aceptación:**
- El resultado debe ser un `DataFrame` con columnas: `categoria`, `ventas_totales`, `ticket_promedio`.
- Debe estar ordenado por `ventas_totales` de mayor a menor.
- Debes mostrar las 5 primeras filas con `.head()`.

**Pistas:**
- Crea una columna auxiliar `venta = precio * unidades`.
- Usa `groupby('categoria').agg(...)` con alias de columnas.
- Para ticket promedio ponderado, usa una razón de sumatorias.

In [ ]:

# TODO: Solución Ejercicio 1

## Ejercicio 2. Segmentación temporal y filtrado avanzado
**Contexto técnico:** Como analista de datos, debes preparar un **subset** de registros para un experimento A/B que solo considera transacciones realizadas **a partir** de una fecha y con **precio** dentro de un rango específico.

**Datos / entradas:** Un `DataFrame` con `fecha` (tipo fecha), `precio` y `categoria`.

**Requerimientos:**
1. Filtrar registros con `fecha >= 2024-01-04`.
2. Mantener solo filas con `precio` en el rango `[20, 200]`.
3. Seleccionar únicamente las columnas `fecha`, `categoria` y `precio`.
4. Mostrar las primeras 3 filas resultantes.

**Criterios de aceptación:**
- El `DataFrame` filtrado solo contiene precios dentro del rango y fechas a partir del umbral.
- Columnas en el orden solicitado.

**Pistas:**
- Recuerda usar paréntesis al combinar condiciones con `&`.
- Usa `loc` para seleccionar columnas por etiqueta.

In [ ]:

# TODO: Solución Ejercicio 2

## Ejercicio 3. Enriquecimiento con transformaciones por grupo
**Contexto técnico:** Como científico/a de datos, necesitas **normalizar** la columna `unidades` dentro de cada `categoria` para comparar el desempeño relativo entre productos, independientemente de su escala.

**Datos / entradas:** `DataFrame` con `categoria` y `unidades` (enteros positivos).

**Requerimientos:**
1. Calcular el **z-score** por categoría sobre `unidades` y añadirlo como columna `z_unidades`.
2. Comprobar que la media por categoría de `z_unidades` está cercana a 0.
3. Mostrar una tabla con `categoria`, `unidades`, `z_unidades` (primeras 5 filas).

**Criterios de aceptación:**
- Nueva columna `z_unidades` creada correctamente.
- Media por categoría de `z_unidades` ≈ 0 (diferencias menores por redondeo).

**Pistas:**
- Usa `groupby('categoria')['unidades'].transform(...)` con una función que aplique z-score.

In [ ]:

# TODO: Solución Ejercicio 3

## Ejercicio 4 (opcional). Exportación del resultado y validaciones mínimas
**Contexto técnico:** Debes entregar a un equipo externo un **extracto** con métricas por categoría. El equipo requiere un archivo **Excel** con nombre y ubicación definidos.

**Datos / entradas:** Resultado del **Ejercicio 1** u otro `DataFrame` agregado.

**Requerimientos:**
1. Validar que no existan valores nulos en las columnas clave.
2. Exportar a Excel sin índice.
3. Confirmar ruta de salida y tamaño del archivo.

**Criterios de aceptación:**
- Archivo `.xlsx` generado en la ruta configurada, sin índice.
- Columnas clave sin nulos.

**Pistas:**
- Usa `notna().all()` para columnas clave.
- Usa `to_excel(ruta, index=False)` y revisa con el sistema de archivos.

In [ ]:

# TODO: Solución Ejercicio 4